<a href="https://colab.research.google.com/github/mygetzu/shopee_product_detection/blob/master/shopee_product_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

!unzip "./gdrive/My Drive/shopee-sentiment-analysis_dataset.zip"

Archive:  ./gdrive/My Drive/shopee-sentiment-analysis_dataset.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


In [1]:
import pandas as pd
import re
from datetime import datetime
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [5]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train

,review_id,review,rating
0,0,Ga disappointed neat products .. Meletot Hilsn...,1
1,1,"Rdtanya replace broken glass, broken chargernya",1
2,2,Nyesel bngt dsni shopping antecedent photo mes...,1
3,3,Sent a light blue suit goods ga want a refund,1
4,4,Pendants came with dents and scratches on its ...,1
...,...,...,...
146806,146806,Excellent product quality delivery speed is ve...,5
146807,146807,thanks gan,5
146808,146808,Awesome awesome quality merchandise value CP ...,5
146809,146809,Nice Packing boxes made effective price .........,5


In [24]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn, stopwords, wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')

class SentimentAnalyzer:
    def __init__(self):
        self.sentimentAnalyzer = SentimentIntensityAnalyzer()
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def preprocess_text(self, text):
        # Special Character Filtering
        text_char_filter = self.specialchar_filtering_text(text)
        # To lower case
        text_lower = self.to_lowercase_text(text_char_filter)
        # Tokenized
        tokenized = self.tokenize_text(text_lower)

        result = tokenized
        return result

    def specialchar_filtering_text(self, text):
        # print("\n======================== Special Char Filtering =========================")
        result = " ".join(re.findall("[a-zA-Z]+", text))
        return result

    def to_lowercase_text(self, text):
        # print("\n======================== Data case folding =========================")
        result = text.lower()
        return result

    def tokenize_text(self, text):
        # print("[", datetime.now(), "] Tokenizing data....")
        result = nltk.pos_tag(word_tokenize(text))
        return result

    def lemmatize_text(self, text, pos_tag):
        result = self.lemmatizer.lemmatize(text, pos_tag)
        # print(result)
        return result

    def get_wordnet_pos_tag(self, tag):
        tag_dict = {
            "J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV,
        }

        return tag_dict.get(tag, wordnet.NOUN)

    def get_vader(self, word):
        return self.sentimentAnalyzer.polarity_scores(word)

    def get_vader_result(self, text):
        text_preprocessed = self.preprocess_text(text)
        vader = self.sentimentAnalyzer.polarity_scores(text_preprocessed)

        sentences_result = 0
        total = 0

        word_count = 0
        for word in text_preprocessed:
            if word[0] not in self.stop_words:
                try:
                    degree = self.get_vader(word)
                    result = self.get_aggregation(
                        degree['pos'], degree['neg'], degree['neu'])
                    # print("Result = ", result)

                    if result != None:
                        word_count += 1

                    sentences_result += result
                except:
                    continue

        if word_count > 0:
            sentences_result = sentences_result / word_count
        else :
            sentences_result = 0

        return (sentences_result - (-1)) / 2


    def get_wordnet_degree(self, word):
        pos_tag = self.get_wordnet_pos_tag(word[1][0])

        lemmatized = self.lemmatize_text(word[0], pos_tag)
        synset = swn.senti_synset('{0}.{1}.03'.format(lemmatized, pos_tag))

        return {
            'positive': synset.pos_score(),
            'negative': synset.neg_score(),
            'objective': synset.obj_score()
        }

    def get_aggregation(self, pos, neg, obj):
        pos_wordnet = 0
        neg_wordnet = 0
        result = 0

        if pos > neg:
            pos_wordnet = pos / (pos + neg)
            result = pos_wordnet - (obj * pos_wordnet)
        elif pos < neg:
            neg_wordnet = neg / (pos + neg) * -1
            result = neg_wordnet - (obj * neg_wordnet)
        else:
            result = 0

        return result

    def get_sentiwordnet(self, text):
        sentences_result = 0
        total = 0

        # print("Review : ", text)

        text_preprocessed = self.preprocess_text(text)

        word_count = 0
        for word in text_preprocessed:
            if word[0] not in self.stop_words:
                try:
                    degree = self.get_wordnet_degree(word)
                    result = self.get_aggregation(
                        degree['positive'], degree['negative'], degree['objective'])
                    # print("Result = ", result)

                    if result != None:
                        word_count += 1

                    sentences_result += result
                except:
                    continue

        # print("[", datetime.now(), "] Word count :", word_count)
        if word_count > 0:
            sentences_result = sentences_result / word_count
        else :
            sentences_result = 0

        # print("[", datetime.now(), "] Sentences result :", sentences_result)
        return (sentences_result - (-1)) / 2

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [25]:
# Test sentiment
sentiment_analyzer = SentimentAnalyzer()
wordnet_score = sentiment_analyzer.get_sentiwordnet(df_train.iloc[0]['review'])
vader_score = sentiment_analyzer.get_vader_score(df_train.iloc[0]['review'])
normalized_wordnet = (wordnet_score - (-1)) / 2
print("Normalized wordnet : ", wordnet_score)
print("Normalized vader : ", vader_score)

AttributeError: ignored

In [13]:
df_train['wordnet_score'] = df_train.apply(lambda x : sentiment_analyzer.get_sentiwordnet(x['review']), axis=1)
df_train['vader_score'] = df_train.apply(lambda x : sentiment_analyzer.get_vader(x['review']), axis=1)
df_train

,review_id,review,rating,wordnet_score,vader_score
0,0,Ga disappointed neat products .. Meletot Hilsn...,1,0.604167,0.4215
1,1,"Rdtanya replace broken glass, broken chargernya",1,0.406250,-0.7351
2,2,Nyesel bngt dsni shopping antecedent photo mes...,1,0.468750,0.8834
3,3,Sent a light blue suit goods ga want a refund,1,0.479167,0.0772
4,4,Pendants came with dents and scratches on its ...,1,0.458333,0.3612
...,...,...,...,...,...
146806,146806,Excellent product quality delivery speed is ve...,5,0.656250,0.8775
146807,146807,thanks gan,5,0.500000,0.4404
146808,146808,Awesome awesome quality merchandise value CP ...,5,0.645833,0.9323
146809,146809,Nice Packing boxes made effective price .........,5,0.572917,0.7096


In [15]:
df_test['wordnet_score'] = df_test.apply(lambda x : sentiment_analyzer.get_sentiwordnet(x['review']), axis=1)
df_test['vader_score'] = df_test.apply(lambda x : sentiment_analyzer.get_vader(x['review']), axis=1)
df_test

,review_id,review,wordnet_score,vader_score
0,1,"Great danger, cool, motif and cantik2 jg model...",0.479167,0.7357
1,2,One of the shades don't fit well,0.541667,-0.4449
2,3,Very comfortable,0.625000,0.5563
3,4,Fast delivery. Product expiry is on Dec 2022. ...,0.482143,-0.6597
4,5,it's sooooo cute! i like playing with the glit...,0.531250,0.9059
...,...,...,...,...
60422,60423,Product has been succesfully ordered and shipp...,0.567308,0.6808
60423,60424,Opening time a little scared. Fear dalemnya de...,0.575000,-0.7331
60424,60425,The product quality is excellent. The origina...,0.610577,0.9390
60425,60426,They 're holding up REALLY well also .,0.687500,0.4812


In [16]:
# Normalization
normalization = preprocessing.MinMaxScaler()
df_train[['wordnet_score', 'vader_score']] = normalization.fit_transform(df_train[['wordnet_score', 'vader_score']])
df_test[['wordnet_score', 'vader_score']] = normalization.fit_transform(df_test[['wordnet_score', 'vader_score']])

X_train = df_train[['wordnet_score', 'vader_score']]
X_test = df_test[['wordnet_score', 'vader_score']]
y_train = df_train['rating']

In [18]:
# KNN Method

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)
y_predict = knn.predict(X_test)
print(df_test)

df_test['rating'] = df_test.apply(lambda x : y_predict[x.name], axis=1)

submission = df_test[['review_id', 'rating']]
submission.to_csv('sample_data/submission_product_sentiment.csv', index=None)

       review_id  ... rating
0              1  ...      5
1              2  ...      1
2              3  ...      4
3              4  ...      1
4              5  ...      5
...          ...  ...    ...
60422      60423  ...      3
60423      60424  ...      1
60424      60425  ...      5
60425      60426  ...      4
60426      60427  ...      3

[60427 rows x 5 columns]


In [21]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
print(df_test)

df_test['rating'] = df_test.apply(lambda x : y_predict[x.name], axis=1)

submission = df_test[['review_id', 'rating']]
submission.to_csv('sample_data/submission_product_sentiment_boost.csv', index=None)

       review_id  ... rating
0              1  ...      3
1              2  ...      1
2              3  ...      3
3              4  ...      1
4              5  ...      5
...          ...  ...    ...
60422      60423  ...      3
60423      60424  ...      1
60424      60425  ...      5
60425      60426  ...      4
60426      60427  ...      5

[60427 rows x 5 columns]
